In [1]:
import pandas as pd

In [2]:
def read_file_as_df(file_name):
    import pandas as pd
    import csv

    import sys
    import pandas as pd

    maxInt = sys.maxsize

    while True:
        # decrease the maxInt value by factor 10
        # as long as the OverflowError occurs.

        try:
            csv.field_size_limit(maxInt)
            break
        except OverflowError:
            maxInt = int(maxInt/10)

    file = []
    col = []

    with open(file_name, encoding='latin-1') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        line_count = 0
        for row in csv_reader:
            if line_count==0:
                for r in row:
                    col.append(r)
                line_count+=1
            else:
                line = []
                for r in row:
                    line.append(r)
                file.append(line)
                line_count += 1


    df = pd.DataFrame(file, columns = col)
    return df

In [15]:
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=289, stratify=y)

In [20]:
from sklearn import metrics

df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

y_train = df_train['Age']
X_train = df_train['req-text']

y_test = df_test['Age']
X_test = df_test['req-text']

tokenizer = Tokenizer()  
tokenizer.fit_on_texts(X_train)

vocab_size = len(tokenizer.word_index) + 1 

sequences = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, maxlen=200)

test_sequences = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(test_sequences, maxlen=200)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=289, stratify=y_train)

y_train = np.array(y_train)
y_test = np.array(y_test) 

model = create_model()

history = model.fit(
        X_train, y_train,
        epochs=20,
        batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping]
)

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

print(metrics.accuracy_score(y_test, y_pred_classes))

Epoch 1/20
224/224 [==============================] - 20s 85ms/step - loss: 1.0236 - accuracy: 0.4459 - val_loss: 0.9406 - val_accuracy: 0.5260
Epoch 2/20
224/224 [==============================] - 18s 80ms/step - loss: 0.8718 - accuracy: 0.5778 - val_loss: 0.9203 - val_accuracy: 0.5562
Epoch 3/20
224/224 [==============================] - 18s 79ms/step - loss: 0.7292 - accuracy: 0.6626 - val_loss: 0.9599 - val_accuracy: 0.5587
Epoch 4/20
224/224 [==============================] - 18s 81ms/step - loss: 0.6340 - accuracy: 0.7105 - val_loss: 1.0198 - val_accuracy: 0.5640
Epoch 5/20
224/224 [==============================] - 18s 79ms/step - loss: 0.5678 - accuracy: 0.7441 - val_loss: 1.0506 - val_accuracy: 0.5655
Epoch 6/20
224/224 [==============================] - 18s 79ms/step - loss: 0.5141 - accuracy: 0.7666 - val_loss: 1.1516 - val_accuracy: 0.5651
Epoch 7/20
224/224 [==============================] - 18s 81ms/step - loss: 0.4755 - accuracy: 0.7876 - val_loss: 1.2840 - val_accuracy:

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()  
tokenizer.fit_on_texts(X_train)

vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding 

In [19]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [12]:
vocab_size

66125

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

sequences = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(sequences, maxlen=200)

test_sequences = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(test_sequences, maxlen=200)

In [10]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

def create_model(input_length = 200, embedding_size=128, lstm_units=4, dropout_rate=0.5, learning_rate=0.01):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=input_length))
    model.add(tf.keras.layers.LSTM(lstm_units, return_sequences=False))
    model.add(tf.keras.layers.Dropout(dropout_rate))
    model.add(tf.keras.layers.Dense(3, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

2025-05-26 20:09:51.813472: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-26 20:09:51.813489: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
model_ = KerasClassifier(build_fn=create_model, epochs=20, batch_size=128)

param_grid = {
    'embedding_size': [128, 256],
    'lstm_units': [4, 16],
    'dropout_rate': [0.3, 0.5],
    'learning_rate': [0.001, 0.01],
}

grid = GridSearchCV(estimator=model_, param_grid=param_grid, cv=5, verbose=0)

grid_result = grid.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=[early_stopping])


In [26]:
grid_result.best_params_, grid_result.best_score_

({'dropout_rate': 0.5,
  'embedding_size': 256,
  'learning_rate': 0.001,
  'lstm_units': 16},
 0.5749231100082397)

In [3]:
dict_class = pd.read_csv('../data/dict-class50.csv', encoding='latin-1')

In [4]:
dict_class

,Unnamed: 0.1,Unnamed: 0,age-bracket,req-text,req-text-clean,Age,glex1,glex2,glex3,c,c50,c5025,cmean,c75,c7525
0,0,34599,a43-xx,Solicito os registro de reclamações feitas a e...,solicito registro reclamações feitas ouvidoria...,2,-0.312707,-0.057745,-1.074704,2,2,-1,-1,-1,-1
1,1,23357,a17-30,"No estado do Amazonas, existem agências da pre...",estado amazonas existem agências previdência s...,0,-0.153447,1.151447,1.144852,1,1,-1,1,-1,-1
2,6,5476,a17-30,"Olá, Estou realizando meu trabalho de conclusã...",olá realizando trabalho conclusão curso estuda...,0,2.274279,3.164122,1.419071,0,0,0,0,0,0
3,10,3721,a43-xx,Gostaria de solicitar dados do consumo de ener...,gostaria solicitar dados consumo energia elétr...,2,3.460562,2.882652,0.433025,0,0,0,-1,-1,0
4,11,23181,a31-42,"Boa tarde, gostaria da quantidade de farmácias...",boa tarde gostaria quantidade farmácias popula...,1,0.929649,2.195696,1.810741,0,0,0,0,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4433,9539,5349,a43-xx,"Prezado(a), Gostaria de obter as seguintes inf...",prezadoa gostaria obter seguintes informações ...,2,2.445181,3.786774,2.567361,0,0,0,0,0,0
4434,9540,46123,a43-xx,Então excelentíssimo Dr. Ouvidor do Governo Fe...,então excelentíssimo dr ouvidor governo federa...,2,-2.008942,-3.061500,-1.501102,2,2,2,2,2,2
4435,9546,30936,a31-42,Dúvidas sobre o sindicato que deveremos fazer ...,dúvidas sobre sindicato deveremos fazer contri...,1,-1.075373,-1.779589,-1.257609,2,2,-1,2,2,-1
4436,9547,38298,a17-30,Venho por meio deste solicitar os seguintes da...,venho meio deste solicitar seguintes dados mon...,0,2.448322,2.200500,0.792049,0,0,0,0,0,0


In [7]:
def get_age_class(age):
    if age == 'a17-30': return 0
    elif age == 'a31-42': return 1
    else: return 2

In [22]:
print(metrics.f1_score(y_test, y_pred_classes, average='macro'))

0.5692163663030197


In [46]:
print(metrics.f1_score(y_test, y_pred_classes, average='weighted'))

0.5703314633567348


In [45]:
model.save('final-model.keras')

In [17]:
import numpy as np

In [23]:
dict_class = pd.read_csv('../data/dict-class50.csv', encoding='latin-1')

In [24]:
dict_class.head()

,Unnamed: 0.1,Unnamed: 0,age-bracket,req-text,req-text-clean,Age,glex1,glex2,glex3,c,c50,c5025,cmean,c75,c7525
0,0,34599,a43-xx,Solicito os registro de reclamações feitas a e...,solicito registro reclamações feitas ouvidoria...,2,-0.312707,-0.057745,-1.074704,2,2,-1,-1,-1,-1
1,1,23357,a17-30,"No estado do Amazonas, existem agências da pre...",estado amazonas existem agências previdência s...,0,-0.153447,1.151447,1.144852,1,1,-1,1,-1,-1
2,6,5476,a17-30,"Olá, Estou realizando meu trabalho de conclusã...",olá realizando trabalho conclusão curso estuda...,0,2.274279,3.164122,1.419071,0,0,0,0,0,0
3,10,3721,a43-xx,Gostaria de solicitar dados do consumo de ener...,gostaria solicitar dados consumo energia elétr...,2,3.460562,2.882652,0.433025,0,0,0,-1,-1,0
4,11,23181,a31-42,"Boa tarde, gostaria da quantidade de farmácias...",boa tarde gostaria quantidade farmácias popula...,1,0.929649,2.195696,1.810741,0,0,0,0,-1,-1


In [31]:
df_test['Predict']=y_pred_classes

In [32]:
df_test['Unnamed: 0'] = df_test['Unnamed: 0'].astype(int)
dict_class['Unnamed: 0'] = dict_class['Unnamed: 0'].astype(int)

compare_test = df_test[df_test['Unnamed: 0'].isin(dict_class['Unnamed: 0'])]

In [33]:
compare_test.shape, dict_class.shape

((4438, 6), (4438, 15))

In [34]:
compare_test = compare_test
compare_dict = dict_class[['Unnamed: 0', 'c50']]
compare = compare_test.merge(compare_dict, left_on='Unnamed: 0', right_on='Unnamed: 0')

In [35]:
compare

,Unnamed: 0,age-bracket,req-text,req-text-clean,Age,Predict,c50
0,34599,a43-xx,Solicito os registro de reclamações feitas a e...,solicito registro reclamações feitas ouvidoria...,2,2,2
1,23357,a17-30,"No estado do Amazonas, existem agências da pre...",estado amazonas existem agências previdência s...,0,0,1
2,5476,a17-30,"Olá, Estou realizando meu trabalho de conclusã...",olá realizando trabalho conclusão curso estuda...,0,1,0
3,3721,a43-xx,Gostaria de solicitar dados do consumo de ener...,gostaria solicitar dados consumo energia elétr...,2,2,0
4,23181,a31-42,"Boa tarde, gostaria da quantidade de farmácias...",boa tarde gostaria quantidade farmácias popula...,1,2,0
...,...,...,...,...,...,...,...
4433,5349,a43-xx,"Prezado(a), Gostaria de obter as seguintes inf...",prezadoa gostaria obter seguintes informações ...,2,2,0
4434,46123,a43-xx,Então excelentíssimo Dr. Ouvidor do Governo Fe...,então excelentíssimo dr ouvidor governo federa...,2,2,2
4435,30936,a31-42,Dúvidas sobre o sindicato que deveremos fazer ...,dúvidas sobre sindicato deveremos fazer contri...,1,1,2
4436,38298,a17-30,Venho por meio deste solicitar os seguintes da...,venho meio deste solicitar seguintes dados mon...,0,0,0


In [39]:
metrics.f1_score(compare['Age'], compare[f'Predict'], average='macro'), metrics.f1_score(compare['Age'], compare['c50'], average='macro')

(0.6137939745747266, 0.6126143766416132)

In [47]:
metrics.f1_score(compare['Age'], compare[f'Predict'], average='weighted'), metrics.f1_score(compare['Age'], compare['c50'], average='weighted')

(0.6166169505131281, 0.6112500490970422)

In [48]:
metrics.accuracy_score(compare['Age'], compare[f'Predict']), metrics.accuracy_score(compare['Age'], compare['c50'])

(0.6169445696259577, 0.6144659756647138)

In [40]:
merged_df = df_test.merge(compare[['Unnamed: 0', 'c50']], on='Unnamed: 0', how='left')

In [41]:
merged_df['Predict'] = merged_df['c50'].combine_first(merged_df['Predict'])

In [42]:
metrics.f1_score(merged_df['Age'], merged_df[f'Predict'], average='macro'), metrics.f1_score(df_test['Age'], df_test['Predict'], average='macro')

(0.5688086361394552, 0.5692163663030197)

In [50]:
metrics.accuracy_score(merged_df['Age'], merged_df[f'Predict']), metrics.accuracy_score(df_test['Age'], df_test['Predict'])

(0.569559300743222, 0.5707107714853973)

In [49]:
metrics.f1_score(merged_df['Age'], merged_df[f'Predict'], average='weighted'), metrics.f1_score(df_test['Age'], df_test['Predict'], average='weighted')

(0.5685760359237693, 0.5703314633567348)

In [44]:
compare.shape[0]/df_test.shape[0]

0.4645661048885167